## Load Libraries

In [1]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils

from functools import partial
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

import os
import time


In [2]:
# Show number of avlaible CPU threads
# With mulithreading this number is twice the number of physical cores
cpu_av = os.cpu_count()
print("Number of available CPU's: {}".format(cpu_av))

Number of available CPU's: 12


In [3]:
# Set the number CPUS that should be used per trial and dataloader
# If set to 1 number of cucurrent training networking is equal to this number
# In case of training with GPU this will be limited to number of models training simultaneously on GPU
# So number of CPU threads for each trial can be increased 
cpus_per_trial = 1
gpus_per_trial = 0

## Load data from files

In [4]:
#Path to data
train_path = 'Data/data_train.npz'
test_path = 'Data/data_test.npz'

In [5]:
train_data = np.load(train_path, allow_pickle=True)
test_data = np.load(test_path, allow_pickle=True)

In [6]:
ClusterN_train = train_data['ClusterN']
Cluster_train = train_data['Cluster']
ClusterTiming_train = train_data['ClusterTiming']
ClusterType_train = train_data['ClusterType']
ClusterE_train = train_data['ClusterE']
ClusterPt_train = train_data['ClusterPt']
ClusterModuleNumber_train = train_data['ClusterModuleNumber']
ClusterCol_train = train_data['ClusterCol']
ClusterRow_train = train_data['ClusterRow']
ClusterM02_train = train_data['ClusterM02']
ClusterM20_train = train_data['ClusterM20']
ClusterDistFromVert_train = train_data['ClusterDistFromVert']
PartE_train = train_data['PartE']
PartPt_train = train_data['PartPt']
PartEta_train = train_data['PartEta']
PartPhi_train = train_data['PartPhi']
PartIsPrimary_train = train_data['PartIsPrimary']
PartPID_train = train_data['PartPID']

In [7]:
ClusterN_test = test_data['ClusterN']
Cluster_test = test_data['Cluster']
ClusterTiming_test = test_data['ClusterTiming']
ClusterType_test = test_data['ClusterType']
ClusterE_test = test_data['ClusterE']
ClusterPt_test = test_data['ClusterPt']
ClusterModuleNumber_test = test_data['ClusterModuleNumber']
ClusterCol_test = test_data['ClusterCol']
ClusterRow_test = test_data['ClusterRow']
ClusterM02_test = test_data['ClusterM02']
ClusterM20_test = test_data['ClusterM20']
ClusterDistFromVert_test = test_data['ClusterDistFromVert']
PartE_test = test_data['PartE']
PartPt_test = test_data['PartPt']
PartEta_test = test_data['PartEta']
PartPhi_test = test_data['PartPhi']
PartIsPrimary_test = test_data['PartIsPrimary']
PartPID_test = test_data['PartPID']

## Temporary change for PID into three categories

In [8]:
def change_pid(arr):
    arr[np.nonzero((arr != 111) & (arr != 221))] = 0
    arr[arr == 111] = 1
    arr[arr == 221] = 2

In [9]:
change_pid(PartPID_test)
change_pid(PartPID_train)

## Reshape the arrays into [size, 1] for usage with ptorch

For linear layers input is expected as [batch_size, num_features] so no need to reshape the existing arrays like Cluster

reconstrcuted clusters later will have to have dim [batch_size, channel, height, width] as input for conv2d-layers

In [10]:
maxClusN_train = np.max(ClusterN_train)
maxClusN_test = np.max(ClusterN_test)
maxClusN = np.max([maxClusN_test, maxClusN_train])

In [11]:
ClusterN_train = ClusterN_train.reshape((ClusterN_train.size, 1))
#Cluster_train = Cluster_train.reshape((ClusterE_train.size, maxClusN))
ClusterType_train = ClusterType_train.reshape((ClusterType_train.size, 1))
ClusterE_train = ClusterE_train.reshape((ClusterE_train.size, 1))
ClusterPt_train = ClusterPt_train.reshape((ClusterPt_train.size, 1))
#ClusterModuleNumber_train = ClusterModuleNumber_train.reshape((ClusterModuleNumber_train.size, maxClusN))
#ClusterRow_train = ClusterRow_train.reshape((ClusterRow_train.size, maxClusN))
#ClusterCol_train = ClusterCol_train.reshape((ClusterCol_train.size, maxClusN))
ClusterM02_train = ClusterM02_train.reshape((ClusterM02_train.size, 1))
ClusterM20_train = ClusterM20_train.reshape((ClusterM20_train.size, 1))
ClusterDistFromVert_train = ClusterDistFromVert_train.reshape((ClusterDistFromVert_train.size, 1))
PartE_train = PartE_train.reshape((PartE_train.size, 1))
PartPt_train = PartPt_train.reshape((PartPt_train.size, 1))
PartEta_train = PartEta_train.reshape((PartEta_train.size, 1))
PartPhi_train = PartPhi_train.reshape((PartPhi_train.size, 1))
PartIsPrimary_train = PartIsPrimary_train.reshape((PartIsPrimary_train.size, 1))
PartPID_train = PartPID_train.reshape((PartPID_train.size, 1))

In [12]:
ClusterN_test = ClusterN_test.reshape((ClusterN_test.size, 1))
#Cluster_test = Cluster_test.reshape((ClusterE_test.size, maxClusN))
ClusterType_test = ClusterType_test.reshape((ClusterType_test.size, 1))
ClusterE_test = ClusterE_test.reshape((ClusterE_test.size, 1))
ClusterPt_test = ClusterPt_test.reshape((ClusterPt_test.size, 1))
#ClusterModuleNumber_test = ClusterModuleNumber_test.reshape((ClusterModuleNumber_test.size, maxClusN))
#ClusterRow_test = ClusterRow_test.reshape((ClusterRow_test.size, maxClusN))
#ClusterCol_test = ClusterCol_test.reshape((ClusterCol_test.size, maxClusN))
ClusterM02_test = ClusterM02_test.reshape((ClusterM02_test.size, 1))
ClusterM20_test = ClusterM20_test.reshape((ClusterM20_test.size, 1))
ClusterDistFromVert_test = ClusterDistFromVert_test.reshape((ClusterDistFromVert_test.size, 1))
PartE_test = PartE_test.reshape((PartE_test.size, 1))
PartPt_test = PartPt_test.reshape((PartPt_test.size, 1))
PartEta_test = PartEta_test.reshape((PartEta_test.size, 1))
PartPhi_test = PartPhi_test.reshape((PartPhi_test.size, 1))
PartIsPrimary_test = PartIsPrimary_test.reshape((PartIsPrimary_test.size, 1))
PartPID_test = PartPID_test.reshape((PartPID_test.size, 1))

## Load it to pytorch `tensor`

In [13]:
ClusterN_train = torch.as_tensor(ClusterN_train, dtype=torch.uint8)
Cluster_train = torch.as_tensor(Cluster_train, dtype=torch.float32)
ClusterTiming_train = torch.as_tensor(ClusterTiming_train, dtype=torch.float32)
ClusterType_train = torch.as_tensor(ClusterType_train, dtype=torch.uint8)
ClusterE_train = torch.as_tensor(ClusterE_train, dtype=torch.float32)
ClusterPt_train = torch.as_tensor(ClusterPt_train, dtype=torch.float32)
ClusterModuleNumber_train = torch.as_tensor(ClusterModuleNumber_train, dtype=torch.uint8)
ClusterRow_train = torch.as_tensor(ClusterRow_train, dtype=torch.uint8)
ClusterCol_train = torch.as_tensor(ClusterCol_train, dtype=torch.uint8)
ClusterM02_train = torch.as_tensor(ClusterM02_train, dtype=torch.float32)
ClusterM20_train = torch.as_tensor(ClusterM20_train, dtype=torch.float32)
ClusterDistFromVert_train = torch.as_tensor(ClusterDistFromVert_train, dtype=torch.float32)
PartE_train = torch.as_tensor(PartE_train, dtype=torch.float32)
PartPt_train = torch.as_tensor(PartPt_train, dtype=torch.float32)
PartEta_train = torch.as_tensor(PartEta_train, dtype=torch.float32)
PartPhi_train = torch.as_tensor(PartPhi_train, dtype=torch.float32)
PartIsPrimary_train = torch.as_tensor(PartIsPrimary_train, dtype=torch.bool)
PartPID_train = torch.as_tensor(PartPID_train, dtype=torch.short)

In [14]:
ClusterN_test = torch.as_tensor(ClusterN_test, dtype=torch.uint8)
Cluster_test = torch.as_tensor(Cluster_test, dtype=torch.float32)
ClusterTiming_test = torch.as_tensor(ClusterTiming_test, dtype=torch.float32)
ClusterType_test = torch.as_tensor(ClusterType_test, dtype=torch.uint8)
ClusterE_test = torch.as_tensor(ClusterE_test, dtype=torch.float32)
ClusterPt_test = torch.as_tensor(ClusterPt_test, dtype=torch.float32)
ClusterModuleNumber_test = torch.as_tensor(ClusterModuleNumber_test, dtype=torch.uint8)
ClusterRow_test = torch.as_tensor(ClusterRow_test, dtype=torch.uint8)
ClusterCol_test = torch.as_tensor(ClusterCol_test, dtype=torch.uint8)
ClusterM02_test = torch.as_tensor(ClusterM02_test, dtype=torch.float32)
ClusterM20_test = torch.as_tensor(ClusterM20_test, dtype=torch.float32)
ClusterDistFromVert_test = torch.as_tensor(ClusterDistFromVert_test, dtype=torch.float32)
PartE_test = torch.as_tensor(PartE_test, dtype=torch.float32)
PartPt_test = torch.as_tensor(PartPt_test, dtype=torch.float32)
PartEta_test = torch.as_tensor(PartEta_test, dtype=torch.float32)
PartPhi_test = torch.as_tensor(PartPhi_test, dtype=torch.float32)
PartIsPrimary_test = torch.as_tensor(PartIsPrimary_test, dtype=torch.bool)
PartPID_test = torch.as_tensor(PartPID_test, dtype=torch.short)

## Define Functions for DataSet and Dataloader

In [15]:
def load_data_train():
    dataset_train = utils.TensorDataset( ClusterN_train, Cluster_train, ClusterTiming_train, ClusterType_train
                                    , ClusterE_train, ClusterPt_train, ClusterModuleNumber_train
                                    , ClusterRow_train, ClusterCol_train, ClusterM02_train, ClusterM20_train
                                    , ClusterDistFromVert_train, PartE_train, PartPt_train
                                    , PartEta_train, PartPhi_train, PartIsPrimary_train, PartPID_train )
    
    return dataset_train

def load_data_test():
    dataset_test = utils.TensorDataset( ClusterN_test, Cluster_test, ClusterTiming_test, ClusterType_test
                                   , ClusterE_test, ClusterPt_test, ClusterModuleNumber_test
                                   , ClusterRow_test, ClusterCol_test, ClusterM02_test, ClusterM20_test
                                   , ClusterDistFromVert_test, PartE_test, PartPt_test
                                    , PartEta_test, PartPhi_test, PartIsPrimary_test, PartPID_test )
    
    return dataset_test

In [16]:
def get_dataloader(train_ds, val_ds, bs):
    return (
        utils.DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=cpus_per_trial),
        utils.DataLoader(val_ds, batch_size=bs * 2, shuffle=True, num_workers=cpus_per_trial),
    )

## Instance Noise

In [17]:
# https://arxiv.org/abs/1610.04490
INSTANCE_NOISE = False

def add_instance_noise(data, std=0.01):
    return data + torch.distributions.Normal(0, std).sample(data.shape) #.to(device)

## Define the network

In [18]:
class CNN(nn.Module):
    def __init__(self, l1=100, l2=50, l3=25):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1,10, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(10,10, kernel_size=3,  padding=2)
        self.conv3 = nn.Conv2d(10,10, kernel_size=3, padding=0)
        self.conv4 = nn.Conv2d(10,5, kernel_size=1, padding=0)
        self.conv5 = nn.Conv2d(5,3, kernel_size=1)
        self.flatten = nn.Flatten()
        self.dense_nn = nn.Sequential(
            nn.Linear(4005, l1),
            nn.ReLU(),
            nn.Linear(l1, l1),
            nn.ReLU(),
            nn.Linear(l1, l2),
            nn.ReLU(),
            nn.Linear(l2, l3),
            nn.ReLU(),
            nn.Linear(l3,3),
            nn.ReLU()
        )
        
    def forward(self, cluster, clusNumXYEPt):
        cluster = F.relu(self.conv1(cluster))
        cluster = F.relu(self.conv2(cluster))
        cluster = F.relu(self.conv3(cluster))
        cluster = F.relu(self.conv3(cluster))
        x = self.flatten(cluster)
        x = torch.cat([x, clusNumXYEPt], dim=1)
        logits = self.dense_nn(x)
        return logits

## Function for cluster reconstruction

In [19]:
def reconstruct_cluster(ncell, modnum, row, col, data, arrsize=20):
    if not torch.all( modnum[0] == modnum[:ncell]):
        ModNumDif = modnum - torch.min(modnum[:ncell])
        mask = torch.where(ModNumDif == 1)
        col[mask] += 48
        mask = torch.where(ModNumDif == 2)
        row[mask] += 24
        mask = torch.where(ModNumDif == 3)
        row[mask] += 24
        col[mask] += 48

    arr = torch.zeros((arrsize,arrsize), dtype=torch.float32)
  
    col_min = torch.min(col[:ncell])
    row_min = torch.min(row[:ncell])
    width = torch.max(col[:ncell]) - col_min
    height = torch.max(row[:ncell]) - row_min
    offset_h = ((arrsize-height)/2).int()
    offset_w = ((arrsize-width)/2).int()
    
    for i in range(ncell):
        arr[ row[i] - row_min + offset_h, col[i] - col_min + offset_w ] = data[i]

    return arr

## Implement train and validation loop
[0: 'ClusterN', 1:'Cluster', 2:'ClusterTiming', 3:'ClusterType', 4:'ClusterE', 5:'ClusterPt', 6:'ClusterModuleNumber', 7:'ClusterRow', 8:'ClusterCol', 9:'ClusterM02', 10:'ClusterM20', 11:'ClusterDistFromVert', 12:'PartE', 13:'PartPt', 14:'PartEta', 15:'PartPhi', 16:'PartIsPrimary', 17:'PartPID']

In [20]:
def train_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):
    size = len(dataloader.dataset)
    running_loss = 0.0
    epoch_steps = 0
    for batch, data in enumerate(dataloader):
        ClN, Cl, ClE, ClPt, ClModNum, ClRow, ClCol, ClM02, ClM20, ClDist, PartPID = data[0], data[1],\
        data[4], data[5], data[6], \
        data[7], data[8], data[9], data[10], \
        data[11], data[17]
        
        # reconstruct clusters from data
        clusters_e = []

        for i in range(ClN.shape[0]):
            cluster_e = reconstruct_cluster(ClN[i], ClModNum[i], ClRow[i], ClCol[i], Cl[i])
            clusters_e.append(cluster_e)
        
        clusters_e = torch.stack(clusters_e)
        clusters_e = clusters_e.view(-1, 1, 20,20)
        if INSTANCE_NOISE:
            clusters_e = add_instance_noise(clusters_e)
        clusters_e.to(device)
        
        ClusterProperties = torch.cat([ClE, ClPt, ClM02, ClM20, ClDist], dim=1)
        ClusterProperties.to(device)
               
        # zero parameter gradients
        optimizer.zero_grad()
        
        # prediction and loss
        pred = model(clusters_e, ClusterProperties)
        loss = loss_fn(pred, PartPID[:,0].long())
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        epoch_steps += 1
        
        if batch % 10 == 9:
            print("[%d, %5d] loss: %.3f" % (epoch + 1, batch + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0        

In [21]:
def val_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    size = len(dataloader.dataset)
    
    for batch, data in enumerate(dataloader):
        with torch.no_grad():
            ClN, Cl, ClE, ClPt, ClModNum, ClRow, ClCol, ClM02, ClM20, ClDist, PartPID = data[0], data[1],\
            data[4], data[5], data[6], \
            data[7], data[8], data[9], data[10], \
            data[11], data[17]
        
            # reconstruct clusters from data
            clusters_e = []

            for i in range(ClN.shape[0]):
                cluster_e = reconstruct_cluster(ClN[i], ClModNum[i], ClRow[i], ClCol[i], Cl[i])
                clusters_e.append(cluster_e)
        
            clusters_e = torch.stack(clusters_e)
            clusters_e = clusters_e.view(-1, 1, 20,20)
            if INSTANCE_NOISE:
                clusters_e = add_instance_noise(clusters_e)
            clusters_e.to(device)
        
            ClusterProperties = torch.cat([ClE, ClPt, ClM02, ClM20, ClDist], dim=1)
            ClusterProperties.to(device)
            
            pred = model(clusters_e, ClusterProperties)
            correct += (pred.argmax(1) == PartPID[:,0]).type(torch.float).sum().item()

            loss = loss_fn(pred, PartPID[:,0].long())#.item()
            val_loss += loss.cpu().numpy()
            val_steps += 1
    
    with tune.checkpoint_dir(epoch) as checkpoint_dir:
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save((model.state_dict(), optimizer.state_dict()), path)
        
    tune.report(loss=(val_loss / val_steps), accuracy= correct / size)

## Implement training routine

In [22]:
def train_model(config, checkpoint_dir=None):
    
    # load model
    model = CNN(config["l1"],config["l2"],config["l3"])
    
    # check for avlaible resource and initialize device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
    # send model to device
    model.to(device)
    
    # initialise loss function and opptimizer
    loss_fn = F.cross_entropy
    optimizer = torch.optim.Adam(model.parameters(),lr=config["lr"])
    
    # check whether checkpoint is available
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    # load dataset
    dataset_train = load_data_train()
    
    # split trainset in train and validation subsets
    test_abs = int(len(dataset_train) * 0.8)
    subset_train, subset_val = utils.random_split(
        dataset_train, [test_abs, len(dataset_train) - test_abs])
    
    # get dataloaders 
    dataloader_train, dataloader_val = get_dataloader(subset_train, subset_val, int(config["batch_size"]))
                                                      
    for epoch in range(100):
        train_loop(epoch, dataloader_train, model, loss_fn, optimizer, device=device)
        val_loop(epoch, dataloader_train, model, loss_fn, optimizer, device=device)                                              
    
    print("Finished Training")

## Implement method for accuracy testing on test set

In [23]:
def test_accuracy(model, device="cpu"):
    
    dataset_test = load_data_test()
    
    dataloader_test = utils.DataLoader(
        dataset_test, batch_size=4, shuffle=False, num_workers=2)
    
    correct = 0
    total = len(dataloader_test.dataset)
    with torch.no_grad():
        for data in dataloader_test:
            ClN, Cl, ClE, ClPt, ClModNum, ClRow, ClCol, ClM02, ClM20, ClDist, PartPID = data[0], data[1],\
            data[4], data[5], data[6], \
            data[7], data[8], data[9], data[10], \
            data[11], data[17]
        
            # reconstruct clusters from data
            clusters_e = []

            for i in range(ClN.shape[0]):
                cluster_e = reconstruct_cluster(ClN[i], ClModNum[i], ClRow[i], ClCol[i], Cl[i])
                clusters_e.append(cluster_e)
        
            clusters_e = torch.stack(clusters_e)
            clusters_e = clusters_e.view(-1, 1, 20,20)
            if INSTANCE_NOISE:
                clusters_e = add_instance_noise(clusters_e)
            clusters_e.to(device)
        
            ClusterProperties = torch.cat([ClE, ClPt, ClM02, ClM20, ClDist], dim=1)
            ClusterProperties.to(device)
            
            pred = model(clusters_e, ClusterProperties)
            correct += (pred.argmax(1) == PartPID[:,0]).type(torch.float).sum().item()

    return correct / total

## Setup all Ray Tune functionality and start training

In [24]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 10)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 10)),
        "l3": tune.sample_from(lambda _: 2 ** np.random.randint(2, 10)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16, 32, 64])
    }

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    reporter = CLIReporter(
        parameter_columns=["l1", "l2", "l3", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    #Get Current date and time
    timestr = time.strftime("%Y_%m_%d-%H:%M:%S")
    name = "ASHA-" + timestr
    
    result = tune.run(
        partial(train_model),
        name = name,
        resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        local_dir = "./Ray_Results",
        scheduler=scheduler,
        progress_reporter=reporter)
    
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))
    
    best_trained_model = CNN(best_trial.config["l1"], best_trial.config["l2"], best_trial.config["l3"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)
    
    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)
    
    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))

In [25]:
main(num_samples=10, max_num_epochs=10, gpus_per_trial=gpus_per_trial)

2021-09-15 15:03:41,909	ERROR services.py:1272 -- Failed to start the dashboard: Failed to start the dashboard, return code 0. The last 10 lines of /tmp/ray/session_2021-09-15_15-03-40_938529_215227/logs/dashboard.log:
  File "/home/jhonerma/anaconda3/envs/cnn-env/lib/python3.9/site-packages/ray/new_dashboard/dashboard.py", line 207, in <module>
    dashboard = Dashboard(
  File "/home/jhonerma/anaconda3/envs/cnn-env/lib/python3.9/site-packages/ray/new_dashboard/dashboard.py", line 98, in __init__
    raise ex
  File "/home/jhonerma/anaconda3/envs/cnn-env/lib/python3.9/site-packages/ray/new_dashboard/dashboard.py", line 89, in __init__
    build_dir = setup_static_dir()
  File "/home/jhonerma/anaconda3/envs/cnn-env/lib/python3.9/site-packages/ray/new_dashboard/dashboard.py", line 42, in setup_static_dir
    raise FrontendNotFoundError(
FrontendNotFoundError: [Errno 2] Dashboard build directory not found. If installing from source, please follow the additional steps required to build th

== Status ==
Memory usage on this node: 6.2/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/12 CPUs, 0/0 GPUs, 0.0/6.22 GiB heap, 0.0/3.11 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_15-15:03:40
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+------+------+------+-------------+--------------+
| Trial name          | status   | loc   |   l1 |   l2 |   l3 |          lr |   batch_size |
|---------------------+----------+-------+------+------+------+-------------+--------------|
| DEFAULT_5a715_00000 | RUNNING  |       |  128 |    8 |   16 | 0.0128276   |            2 |
| DEFAULT_5a715_00001 | PENDING  |       |    4 |   16 |   16 | 0.00042926  |           16 |
| DEFAULT_5a715_00002 | PENDING  |       |   16 |    4 |  128 | 0.0111267   |            8 |
| DEFAULT_5a715_00003 | PENDING  |       |   32 |    4 

(pid=215346) [5,    10] loss: 0.775
Result for DEFAULT_5a715_00000:
  accuracy: 0.21768707482993196
  date: 2021-09-15_15-03-47
  done: true
  experiment_id: f06570d4f3fa425e84b4d0e4a570bb8f
  hostname: jhonerma-tuxedo
  iterations_since_restore: 1
  loss: 1.0986123085021973
  node_ip: 10.67.95.37
  pid: 215340
  should_checkpoint: true
  time_since_restore: 2.957535743713379
  time_this_iter_s: 2.957535743713379
  time_total_s: 2.957535743713379
  timestamp: 1631711027
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5a715_00000
  
(pid=215346) [6,    10] loss: 0.840
(pid=215346) [7,    10] loss: 0.823
== Status ==
Memory usage on this node: 6.3/15.1 GiB
Using AsyncHyperBand: num_stopped=7
Bracket: Iter 8.000: -0.82437162399292 | Iter 4.000: -0.824166116118431 | Iter 2.000: -0.8172131359577179 | Iter 1.000: -0.9577536648995169
Resources requested: 3.0/12 CPUs, 0/0 GPUs, 0.0/6.22 GiB heap, 0.0/3.11 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/A

2021-09-15 15:03:50,668	INFO tune.py:561 -- Total run time: 7.82 seconds (7.64 seconds for the tuning loop).


Result for DEFAULT_5a715_00008:
  accuracy: 0.7142857142857143
  date: 2021-09-15_15-03-50
  done: true
  experiment_id: 249b14461be9455180d328247076452a
  hostname: jhonerma-tuxedo
  iterations_since_restore: 10
  loss: 0.7618562817573548
  node_ip: 10.67.95.37
  pid: 215344
  should_checkpoint: true
  time_since_restore: 6.328228950500488
  time_this_iter_s: 0.511955738067627
  time_total_s: 6.328228950500488
  timestamp: 1631711030
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 5a715_00008
  
== Status ==
Memory usage on this node: 5.7/15.1 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -0.813855129480362 | Iter 4.000: -0.824166116118431 | Iter 2.000: -0.8172131359577179 | Iter 1.000: -0.9577536648995169
Resources requested: 0/12 CPUs, 0/0 GPUs, 0.0/6.22 GiB heap, 0.0/3.11 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_15-15:03:40
Number of trials: 10/10 (10 TERMINATED)
+---------------------+------------+-------